In [1]:
import importlib as il
import numpy as np

from hilde.parsers import read_structure
from hilde.helpers.supercell import make_cubic_supercell
from hilde.templates.lammps import setup_lammps_si, setup_lammps_gan
from hilde.tasks.calculate import calculate_multiple

In [16]:
ph3 = il.import_module('hilde.phono3py.phono3')

In [17]:
atoms = read_structure('../si.in')

In [18]:
sc2, smatrix2 = make_cubic_supercell(atoms, 216)
sc3, smatrix3 = make_cubic_supercell(atoms, 8)
sc3.inform()


Geometry info for:
  input geometry:    Si8_227_8b
  Symmetry prec.:    1e-05
  Number of atoms:   8
  Species:           Si
  Periodicity:       [ True  True  True]
  Lattice:  
    [5.44 0.   0.  ]
    [0.   5.44 0.  ]
    [0.   0.   5.44]

  Spacegroup:          Fd-3m (227)
  Wyckoff positions:   8*b
  Equivalent atoms:    8*0
  Standard lattice:  
    [5.44 0.   0.  ]
    [0.   5.44 0.  ]
    [0.   0.   5.44]

Cell lengths and angles [Å, °]:
  a, b, c:      5.4400      5.4400      5.4400
  α, β, γ:     90.0000     90.0000     90.0000
  Volume:     160.9892 Å**3
  Tag  1:                supercell
  Tag  2:                ('smatrix', [-1, 1, 1, 1, -1, 1, 1, 1, -1])


In [19]:
phono3py_settings = {
    'atoms': atoms,
    'fc2_supercell_matrix': smatrix2,
    'fc3_supercell_matrix': smatrix3,
    'cutoff_pair_distance': 5
}

In [30]:
il.reload(ph3)
help(ph3.preprocess)

Help on function preprocess in module hilde.phono3py.phono3:

preprocess(atoms, fc2_supercell_matrix, fc3_supercell_matrix, q_mesh=[11, 11, 11], disp=0.03, cutoff_pair_distance=10.0, symprec=1e-05, trigonal=False)
    Creates a phonopy object from given input
    Args:
        atoms: atoms object that represents the (primitive) unit cell
        q_mesh: q-point grid for BZ integrations
        supercell_matrix: supercell matrix
        disp: displacement for the finite displacemt
    
    Returns:
        namedtuple with the phonon3 object, the supercell
        and the supercells_with_displacements as hilde.pAtoms



In [21]:
phonon3, sc2, sc3, scs2, scs3 = ph3.preprocess(**phono3py_settings)

In [22]:
print(f'Number of FC2 displacements: {len(scs2)}')
print(f'Number of FC3 displacements: {len(scs3)}')
print(f'Number of FC3 displacements that were created: {len([sc for sc in scs3 if sc])}')

Number of FC2 displacements: 1
Number of FC3 displacements: 16
Number of FC3 displacements that were created: 16


In [23]:
lammps = setup_lammps_si('.')

## Compute Forces

In [24]:
scs2_computed = calculate_multiple(scs2, lammps, f'{atoms.sysname}/fc2', trajectory='fc2.traj', force=True)
fc2_forces = ph3.get_forces(scs2, scs2_computed)

In [25]:
scs3_computed = calculate_multiple(scs3, lammps, f'{atoms.sysname}/fc3', trajectory='fc3.traj', force=True)
fc3_forces = ph3.get_forces(scs3, scs3_computed)

## Produce Force Constants

In [26]:
phonon3.produce_fc2(fc2_forces)

In [27]:
%time phonon3.produce_fc3(fc3_forces)

Computing fc3[ 1, x, x ] using numpy.linalg.pinv with a displacement:
    [ 0.0300  0.0000  0.0000]
Expanding fc3
Cutting-off fc3 (cut-off distance: 5.000000)
Building atom mapping table...
Creating contracted fc3...
CPU times: user 150 ms, sys: 177 µs, total: 150 ms
Wall time: 145 ms


In [28]:
fc2 = phonon3.get_fc2()
fc3 = phonon3.get_fc3()

In [31]:
phonon3_new = ph3.prepare_phono3py(**phono3py_settings,
                                   fc2=fc2,
                                   fc3=fc3,
                                   q_mesh=3*[11])

In [32]:
phonon3_new.run_thermal_conductivity(write_kappa=True)

========================== Warning ==========================
 Phonon frequency of band index 1 at Gamma is calculated to be 0.015460.
 But this frequency is forced to be zero.
========================== Warning ==========================
 Phonon frequency of band index 2 at Gamma is calculated to be 0.015460.
 But this frequency is forced to be zero.
========================== Warning ==========================
 Phonon frequency of band index 3 at Gamma is calculated to be 0.015460.
 But this frequency is forced to be zero.
-------------------- Lattice thermal conducitivity (RTA) --------------------
Lifetime sampling mesh: [ 11 11 11 ]
======================= Grid point 0 (1/56) =======================
q-point: ( 0.00  0.00  0.00)
Number of triplets: 56
Frequency     group velocity (x, y, z)     |gv|
 k*1  ( 0.00  0.00  0.00)
   0.000   (  -0.000   -0.000   -0.000)    0.000
   0.000   (  -0.000    0.000    0.000)    0.000
   0.000   (   0.000    0.000    0.000)    0.000
  16.538   ( 

======================= Grid point 1 (2/56) =======================
q-point: ( 0.09  0.00  0.00)
Number of triplets: 146
Frequency     group velocity (x, y, z)     |gv|
 k*1  ( 0.09  0.00  0.00)
   0.745   ( -14.530   14.530   14.530)   25.167
   0.745   ( -14.530   14.530   14.530)   25.167
   1.990   ( -39.496   39.496   39.496)   68.409
  16.359   (   7.093   -7.093   -7.093)   12.285
  16.501   (   1.430   -1.430   -1.430)    2.477
  16.501   (   1.430   -1.430   -1.430)    2.477
 k*1  ( 0.09  0.00  0.00)
   0.745   ( -14.530   14.530   14.530)   25.167
   0.745   ( -14.530   14.530   14.530)   25.167
   1.990   ( -39.496   39.496   39.496)   68.409
  16.359   (   7.093   -7.093   -7.093)   12.285
  16.501   (   1.430   -1.430   -1.430)    2.477
  16.501   (   1.430   -1.430   -1.430)    2.477
 k*1  ( 0.09  0.00  0.00)
   0.745   ( -14.530   14.530   14.530)   25.167
   0.745   ( -14.530   14.530   14.530)   25.167
   1.990   ( -39.496   39.496   39.496)   68.409
  16.359   (   7.0

======================= Grid point 2 (3/56) =======================
q-point: ( 0.18  0.00  0.00)
Number of triplets: 146
Frequency     group velocity (x, y, z)     |gv|
 k*1  ( 0.18  0.00  0.00)
   1.440   ( -13.014   13.014   13.014)   22.541
   1.440   ( -13.014   13.014   13.014)   22.541
   3.949   ( -38.498   38.498   38.498)   66.681
  15.837   (  13.577  -13.577  -13.577)   23.517
  16.402   (   2.452   -2.452   -2.452)    4.248
  16.402   (   2.452   -2.452   -2.452)    4.248
 k*1  ( 0.18  0.00  0.00)
   1.440   ( -13.014   13.014   13.014)   22.541
   1.440   ( -13.014   13.014   13.014)   22.541
   3.949   ( -38.498   38.498   38.498)   66.681
  15.837   (  13.577  -13.577  -13.577)   23.517
  16.402   (   2.452   -2.452   -2.452)    4.248
  16.402   (   2.452   -2.452   -2.452)    4.248
 k*1  ( 0.18  0.00  0.00)
   1.440   ( -13.014   13.014   13.014)   22.541
   1.440   ( -13.014   13.014   13.014)   22.541
   3.949   ( -38.498   38.498   38.498)   66.681
  15.837   (  13.5

======================= Grid point 3 (4/56) =======================
q-point: ( 0.27  0.00  0.00)
Number of triplets: 146
Frequency     group velocity (x, y, z)     |gv|
 k*1  ( 0.27  0.00  0.00)
   2.032   ( -10.425   10.425   10.425)   18.057
   2.032   ( -10.425   10.425   10.425)   18.057
   5.828   ( -36.164   36.164   36.164)   62.637
  15.022   (  18.595  -18.595  -18.595)   32.208
  16.268   (   2.739   -2.739   -2.739)    4.745
  16.268   (   2.739   -2.739   -2.739)    4.745
 k*1  ( 0.27  0.00  0.00)
   2.032   ( -10.425   10.425   10.425)   18.057
   2.032   ( -10.425   10.425   10.425)   18.057
   5.828   ( -36.164   36.164   36.164)   62.637
  15.022   (  18.595  -18.595  -18.595)   32.208
  16.268   (   2.739   -2.739   -2.739)    4.745
  16.268   (   2.739   -2.739   -2.739)    4.745
 k*1  ( 0.27  0.00  0.00)
   2.032   ( -10.425   10.425   10.425)   18.057
   2.032   ( -10.425   10.425   10.425)   18.057
   5.828   ( -36.164   36.164   36.164)   62.637
  15.022   (  18.5

======================= Grid point 4 (5/56) =======================
q-point: ( 0.36  0.00  0.00)
Number of triplets: 146
Frequency     group velocity (x, y, z)     |gv|
 k*1  ( 0.36  0.00  0.00)
   2.468   (  -6.796    6.796    6.796)   11.771
   2.468   (  -6.796    6.796    6.796)   11.771
   7.522   ( -30.521   30.521   30.521)   52.864
  14.029   (  20.096  -20.096  -20.096)   34.807
  16.142   (   2.145   -2.145   -2.145)    3.715
  16.142   (   2.145   -2.145   -2.145)    3.715
 k*1  ( 0.36  0.00  0.00)
   2.468   (  -6.796    6.796    6.796)   11.771
   2.468   (  -6.796    6.796    6.796)   11.771
   7.522   ( -30.521   30.521   30.521)   52.864
  14.029   (  20.096  -20.096  -20.096)   34.807
  16.142   (   2.145   -2.145   -2.145)    3.715
  16.142   (   2.145   -2.145   -2.145)    3.715
 k*1  ( 0.36  0.00  0.00)
   2.468   (  -6.796    6.796    6.796)   11.771
   2.468   (  -6.796    6.796    6.796)   11.771
   7.522   ( -30.521   30.521   30.521)   52.864
  14.029   (  20.0

...

======================= Grid point 292 (53/56) =======================
q-point: ( 0.55  0.36  0.18)
Number of triplets: 366
Frequency     group velocity (x, y, z)     |gv|
 k*1  ( 0.55  0.36  0.18)
   3.018   (  -0.000    5.080   12.084)   13.108
   3.555   (   0.000   18.785    7.267)   20.141
   9.280   (   0.000   10.304   46.305)   47.437
  13.137   (  -0.000  -18.382  -30.391)   35.518
  15.599   (  -0.000    0.128   -5.216)    5.217
  15.777   (   0.000    1.597   -8.621)    8.768
 k*1  ( 0.55  0.36  0.18)
   3.018   (   0.000    5.080   12.084)   13.108
   3.555   (  -0.000   18.785    7.267)   20.141
   9.280   (  -0.000   10.304   46.305)   47.437
  13.137   (   0.000  -18.382  -30.391)   35.518
  15.599   (   0.000    0.128   -5.216)    5.217
  15.777   (  -0.000    1.597   -8.621)    8.768
 k*2  (-0.18  0.36  0.18)
   3.018   (  12.084   -5.080   -0.000)   13.108
   3.555   (   7.267  -18.785    0.000)   20.141
   9.280   (  46.305  -10.304    0.000)   47.437
  13.137   ( -3

======================= Grid point 293 (54/56) =======================
q-point: ( 0.64  0.36  0.18)
Number of triplets: 666
Frequency     group velocity (x, y, z)     |gv|
 k*1  ( 0.64  0.36  0.18)
   3.211   (   5.854    1.446   10.278)   11.916
   3.877   (  -1.471    7.779    1.302)    8.023
  10.090   (   7.526    6.691   34.202)   35.654
  12.403   (  -3.459  -10.096  -24.793)   26.993
  15.560   (  -2.708    0.262   -3.154)    4.165
  15.677   (  -1.718    0.331   -6.121)    6.366
 k*2  (-0.27  0.36  0.18)
   3.211   (  10.278   -1.446    5.854)   11.916
   3.877   (   1.302   -7.779   -1.471)    8.023
  10.090   (  34.202   -6.691    7.526)   35.654
  12.403   ( -24.793   10.096   -3.459)   26.993
  15.560   (  -3.154   -0.262   -2.708)    4.165
  15.677   (  -6.121   -0.331   -1.718)    6.366
 k*3  (-0.18  0.45  0.18)
   3.211   (  10.278   -1.446   -5.854)   11.916
   3.877   (   1.302   -7.779    1.471)    8.023
  10.090   (  34.202   -6.691   -7.526)   35.654
  12.403   ( -2

======================= Grid point 304 (55/56) =======================
q-point: ( 0.64  0.45  0.18)
Number of triplets: 366
Frequency     group velocity (x, y, z)     |gv|
 k*1  ( 0.64  0.45  0.18)
   3.400   (  -0.000    7.099    9.965)   12.235
   3.626   (   0.000   12.883   -2.805)   13.185
  10.754   (   0.000   -0.772   41.489)   41.496
  12.040   (  -0.000  -10.695  -35.500)   37.076
  15.486   (  -0.000    1.742   -1.080)    2.049
  15.545   (   0.000    2.249   -4.643)    5.159
 k*1  ( 0.64  0.45  0.18)
   3.400   (   0.000    7.099    9.965)   12.235
   3.626   (  -0.000   12.883   -2.805)   13.185
  10.754   (  -0.000   -0.772   41.489)   41.496
  12.040   (   0.000  -10.695  -35.500)   37.076
  15.486   (   0.000    1.742   -1.080)    2.049
  15.545   (  -0.000    2.249   -4.643)    5.159
 k*2  (-0.27 -0.45  0.18)
   3.400   (   0.000    7.099   -9.965)   12.235
   3.626   (  -0.000   12.883    2.805)   13.185
  10.754   (  -0.000   -0.772  -41.489)   41.496
  12.040   (   

======================= Grid point 305 (56/56) =======================
q-point: (-0.27  0.45  0.18)
Number of triplets: 366
Frequency     group velocity (x, y, z)     |gv|
 k*1  (-0.27  0.45  0.18)
   3.436   (  13.340    5.132    0.000)   14.293
   3.850   (  -9.938    0.564   -0.000)    9.954
  10.873   (  30.085   -8.100    0.000)   31.156
  11.786   ( -24.785    3.775   -0.000)   25.070
  15.507   (  -0.534    0.640    0.000)    0.834
  15.564   (  -4.011    1.190   -0.000)    4.184
 k*1  (-0.27  0.45  0.18)
   3.436   (  13.340    5.132    0.000)   14.293
   3.850   (  -9.938    0.564   -0.000)    9.954
  10.873   (  30.085   -8.100    0.000)   31.156
  11.786   ( -24.785    3.775   -0.000)   25.070
  15.507   (  -0.534    0.640   -0.000)    0.834
  15.564   (  -4.011    1.190   -0.000)    4.184
 k*2  ( 0.45 -0.27  0.18)
   3.436   (   5.132   13.340    0.000)   14.293
   3.850   (   0.564   -9.938   -0.000)    9.954
  10.873   (  -8.100   30.085    0.000)   31.156
  11.786   (   

=================== End of collection of collisions ===================
----------- Thermal conductivity (W/m-k) with tetrahedron method -----------
#  T(K)        xx         yy         zz         yz         xz         xy        #ipm
    0.0      0.000      0.000      0.000      0.000      0.000      0.000 3/7986
   10.0   1936.364   1936.364   1936.364      0.000      0.000     -0.000 3/7986
   20.0    492.952    492.952    492.952      0.000      0.000     -0.000 3/7986
   30.0    262.039    262.039    262.039      0.000      0.000     -0.000 3/7986
   40.0    167.204    167.204    167.204      0.000      0.000     -0.000 3/7986
   50.0    120.266    120.266    120.266      0.000      0.000     -0.000 3/7986
   60.0     93.760     93.760     93.760      0.000      0.000     -0.000 3/7986
   70.0     77.184     77.184     77.184      0.000      0.000     -0.000 3/7986
   80.0     65.941     65.941     65.941      0.000      0.000     -0.000 3/7986
   90.0     57.802     57.802     57.